In [2]:
import json
import re
from tqdm import tqdm

In [4]:
with open("저출산_news.json") as f:
    news_list = json.load(f)

In [5]:
len(news_list)


15005

In [6]:
documents = []
invalid_line_patterns = re.compile(
    r"무단\s*전재|배포\s*금지|Copyrights|관련기사|기사\s*제보|여러분의 제보|카카오톡\s*:s*"
)

for article in tqdm(news_list, mininterval=1):
    text = article["text"]

    lines = text.split("\n")
    filtered_lines = []

    for line in lines:
        line = " ".join(line.split())

        # OOO 기자 패턴 제거
        line = re.sub(r"\w+ 기자", "", line)
        # email 주소 제거
        line = re.sub(r"\w+@\w+\.\w+", "", line)
        # 뉴스에서 []는 [논산] [대구=뉴시스] 등 다양한 참조를 의미하므로 제거
        line = re.sub(r"\[.*\]", "", line)
        # (서울=연합뉴스) 같은 패턴 제거
        line = re.sub(r"\(.*=.*\)", "", line)
        # 2024.10.29/뉴스1 같은 패턴 제거
        line = re.sub(r"\d{4}\.\d{2}\.\d{2}/.*\b", "", line)

        # 공백 제거
        line = " ".join(line.split())

        if invalid_line_patterns.search(line):
            # 무단전재, 배포금지 등이 포함된 문장 이후 문장들은 제외
            break

        # 한국어가 50자 이상 포함된 경우만 포함
        num_korean_chars = len(re.findall(r"[ㄱ-ㅎ가-힣]", line))
        if num_korean_chars >= 10:
            filtered_lines.append(line)

    text = "\n".join(filtered_lines)
    # 한국어가 10자 이상 포함된 경우만 포함
    num_korean_chars = len(re.findall(r"[ㄱ-ㅎ가-힣]", text))
    if num_korean_chars >= 50:
        documents.append(text)

# 중복 제거
documents = list(set(documents))

100%|██████████| 15005/15005 [00:10<00:00, 1413.30it/s]


In [7]:
len(documents)

14688

In [8]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
from collections import Counter

kiwi = Kiwi()  # model_type="sbg"를 넣어주면 더 빠르지만 속도는 느림
stopwords = Stopwords()

document_tokens_list = []
word_freqs = Counter()

for document in tqdm(documents, mininterval=1):
    tokens = []
    for token in kiwi.tokenize(document, stopwords=stopwords):
        tokens.append(token.form)

    if len(tokens) >= 10:
        document_tokens_list.append((tokens, document))
        word_freqs.update(tokens)

100%|██████████| 14688/14688 [10:23<00:00, 23.56it/s]


In [9]:
len(document_tokens_list)

14688

In [10]:
!pip install gensim

In [12]:
from gensim.models.word2vec import Word2Vec

dimension = 128

word2vec = Word2Vec(
    sentences= [tokens for tokens, _ in document_tokens_list ],
    vector_size= dimension,
    min_count=10,
    sg=1, #1= skipgram, 0 cbow
    workers=4,


)

In [13]:
len(word2vec.wv.key_to_index) # access word vocab in dict

21147

In [16]:
word2vec.wv["저출산"]
#word2vec.wv["저출산"].shape

array([-0.01074278,  0.02999723, -0.18548243, -0.33114284,  0.07633556,
       -0.16387568,  0.25494066, -0.31849405,  0.18774448,  0.30227596,
        0.73758423,  0.40865338, -0.12636493, -0.12528154, -0.25089285,
        0.36183125,  0.1300188 ,  0.07122357, -0.7468397 ,  0.40879378,
       -0.1996291 ,  0.20141904, -0.02971657, -0.10598037, -0.22458093,
       -0.54934764, -0.289838  , -0.19555657, -0.027772  , -0.06097885,
       -0.05427239, -0.05740544, -0.06241183,  0.16898757, -0.08444401,
       -0.2564772 ,  0.5258142 , -0.32563147, -0.03493746,  0.1345334 ,
        0.156945  , -0.13101344, -0.0103999 , -0.2678533 ,  0.23818369,
        0.14599407,  0.01706969, -0.12812734,  0.0981571 ,  0.2998148 ,
        0.38071328,  0.03778154, -0.07086729, -0.19562958,  0.09652046,
        0.09193648,  0.00637106, -0.2791668 ,  0.26384073, -0.01733249,
        0.21457589,  0.13870017, -0.20886326, -0.04890452,  0.34591174,
       -0.35744852,  0.4690829 ,  0.19381073,  0.36056975, -0.30

In [15]:
word2vec.wv.similar_by_word("저출산", topn=10)
#to print more related word = sim.log(f(w))

[('출산', 0.6769356727600098),
 ('저', 0.6587146520614624),
 ('출생', 0.6234090328216553),
 ('슈링코', 0.566595733165741),
 ('절벽', 0.5633752942085266),
 ('존망', 0.549551248550415),
 ('맞닥뜨리', 0.5476347208023071),
 ('닥쳐오', 0.5415179133415222),
 ('인구', 0.5412551760673523),
 ('초저', 0.5407400131225586)]

In [ ]:
word2vec.wv.similarity("남자", "여자") 

0.7712673

In [ ]:
import numpy as np

w1 = word2vec.wv["남자"]
w2 = word2vec.wv["여자"]

np.dot(w1,w2)/ (np.linalg.norm(w1) * np.linalg.norm(w2))  #calc similar use dot in np instead of similiraty fucntion

0.77126724

In [21]:
word2vec.wv.most_similar(positive=["서울", "수도권"], negative=["지방"], topn=30) #서울+수도권-지방

[('마포', 0.5787989497184753),
 ('非', 0.573103129863739),
 ('노원구', 0.5678924918174744),
 ('강남', 0.5581784248352051),
 ('서초', 0.5544784069061279),
 ('강북', 0.5478295683860779),
 ('삼성동', 0.5295096039772034),
 ('역삼동', 0.5230748653411865),
 ('동북', 0.5224855542182922),
 ('포 시즌스', 0.5189433693885803),
 ('반포동', 0.5167939066886902),
 ('마포구', 0.5112237334251404),
 ('청량리', 0.5096234083175659),
 ('아파트값', 0.5043401122093201),
 ('성동구', 0.5034322738647461),
 ('영등포구', 0.5033293962478638),
 ('그린벨트', 0.5031585097312927),
 ('송파', 0.5022258162498474),
 ('수도', 0.5014170408248901),
 ('관악구', 0.5006145238876343),
 ('성수동', 0.4954698085784912),
 ('둔촌동', 0.49092280864715576),
 ('노원', 0.4894281327724457),
 ('대치동', 0.48497408628463745),
 ('페어몬트', 0.48280277848243713),
 ('성북', 0.48232439160346985),
 ('SKY', 0.4818992614746094),
 ('동대문', 0.4779307246208191),
 ('938', 0.4744759798049927),
 ('영등포', 0.47361254692077637)]

In [22]:
word2vec.wv.most_similar(negative=["서울", "수도권"], positive=["지방"], topn=30) #서울+수도권-지방

[('고취', 0.17276974022388458),
 ('답례', 0.1695655733346939),
 ('공무원', 0.16383759677410126),
 ('모범', 0.15845024585723877),
 ('자치', 0.1528691202402115),
 ('공적', 0.1483006477355957),
 ('서식', 0.1480470895767212),
 ('ICAO', 0.14490322768688202),
 ('근거', 0.14221864938735962),
 ('사회복무요원', 0.13389843702316284),
 ('권장', 0.13269120454788208),
 ('작가', 0.1319035440683365),
 ('환류', 0.1296674907207489),
 ('참고', 0.12859368324279785),
 ('답안', 0.12664060294628143),
 ('시책', 0.12531882524490356),
 ('각종', 0.1250455230474472),
 ('사명감', 0.12476617097854614),
 ('농', 0.12219865620136261),
 ('명사', 0.12147796154022217),
 ('기본법', 0.11995638161897659),
 ('엄정', 0.11991570144891739),
 ('감시', 0.11805800348520279),
 ('회원', 0.11759807914495468),
 ('규율', 0.11484363675117493),
 ('청렴', 0.1120961457490921),
 ('담', 0.11198306828737259),
 ('규정', 0.11176791787147522),
 ('묘', 0.1108064278960228),
 ('어기', 0.10970661789178848)]

In [ ]:
#if try another word
word2vec.wv["다야나"] #will not exist = out of vocab oov
#so use the fasttext

In [24]:
from gensim.models import FastText

dimension = 128

fasttext = FastText(
    sentences=[tokens for tokens,  _ in document_tokens_list],
    vector_size= dimension,
    min_count= 10,
    sg=1,
    workers=4
    #can use window to see word next to the word. default = 5. so to right 5 word to left 5 word
)

In [ ]:
fasttext.wv["다야나"] #cut vector by ngram 

In [26]:
fasttext.wv.most_similar(positive=["사울", "수도권"], negative=["지방"])

[('마포', 0.46056365966796875),
 ('청량리역', 0.4079980254173279),
 ('오피스텔', 0.4032866954803467),
 ('두정역', 0.3830896317958832),
 ('청량리', 0.38036826252937317),
 ('양산', 0.3630828559398651),
 ('지로', 0.3616540729999542),
 ('오산역', 0.35978958010673523),
 ('홍대', 0.35476651787757874),
 ('구리토평', 0.3532838225364685)]

In [27]:
#doc2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

dimension = 128

doc2vec = Doc2Vec(
    documents=[
        TaggedDocument(words=tokens, tags=[str(i)])
        for i, (tokens, document) in enumerate(document_tokens_list)
    ],
    vector_size= dimension,
    min_count =10,
    workers = 4,

)

In [28]:
doc2vec.wv.most_similar(positive=["강남구", "집값"])

[('아파트값', 0.5790503621101379),
 ('강북', 0.5638124346733093),
 ('0.55', 0.5513043999671936),
 ('전셋값', 0.5451037287712097),
 ('송파구', 0.5309735536575317),
 ('성동구', 0.5257106423377991),
 ('오피스텔', 0.5171188116073608),
 ('성수동', 0.5096216201782227),
 ('영등포구', 0.5049877762794495),
 ('아파트', 0.49892622232437134)]

In [29]:
top_documents = doc2vec.dv.similar_by_vector(
    doc2vec.wv.get_mean_vector(["강남구", "집값"])
)

for doc_index, similarity in top_documents:
    doc_index = int(doc_index)
    raw_document = document_tokens_list[doc_index][1]

    print(raw_document[:200])
    print()

“최근 아파트 매매 거래량이 증가하고 있고 서울 강남 3구(강남·서초·송파)의 집값이 전고점의 95% 수준까지 올라오는 등 회복기에 들어선 것으로 판단됩니다. 서울의 경우 1990년대생들이 주목하는 뉴타운 재개발 단지를 중심으로 집값이 더 상승할 것으로 예상합니다. 다만 교통과 일자리, 학군 등에 따른 집값 양극화 현상은 점점 더 심화할 것으로 보여 상급지

'저출생 위기' 서울 주택공급 늘리려면…"빌라·주택 지역 재개발해야"
"청년 소유할 수 있는 신규아파트 없이 백약이 무효"
"신혼부부 대기자 명부 활용해 적정 주택 매칭해야"
= 저출생 위기 속 주거문제 해결을 위해서는 빌라나 주택 지역을 아파트 중심으로 전환하는 재개발이 활발하게 이뤄져야 한다는 전문가 진단이 나왔다.
27일 서울시청 다목적홀에서 서울시가

서울 떠나는 주된 요인 '주택문제'
경기·인천 인구 유입 전년 동기 대비↑
지난 5개월 간 탈(脫)서울 한 사람(순유출)이 1만593명으로 집계됐다. 이는 전국 17개 시·도 중 가장 많고 전년 같은 기간(7152명) 대비 48.1% 늘어난 수치다. 반면 경기도·인천으로 새 둥지를 튼 사람(순유입)은 전년 같은 기간 대비 각각 14.4%, 16.2% 늘었다

［월요신문=］인천시가 저출생 육아지원 정책을 잇따라 내놓으면서 서울과 경기에서 이사 오는 수요를 더욱 촉발할지 주목받고 있다.
18일 업계에 따르면 인천시는 아이를 낳는 인천시민에게 1억원을 지원하는 사업에 이어 신혼(예비)부부에게 하루 1000원꼴인 임대료의 주택을 빌려주는 저출생 정책을 마련, 내년부터 시행하겠다고 밝혔다.
이른바 '천원주택'은 매입임대

■ 대출요건 완화에 중소형 인기
이번달부터 연간 부부합산 소득
등촌 아이파크 84㎡ 약1억 껑충
광명 롯데캐슬 등 분양도 ‘눈길’
최근 정부가 신생아 특례대출의 소득 요건을 완화하면서 대출 요건에 해당하는 9억 원 이하, 전용면적 85㎡ 이하 아파트의 인기가 더욱 높아질 전망이다.
지난달 19일 국토교통부는 저출산 대책의 일환으로 부부 합산 

In [30]:
doc2vec.dv["1"]

array([ 1.1223933 , -0.6414684 , -0.6024949 ,  1.1877841 , -0.7804724 ,
       -0.51899403, -2.8375735 , -1.1868792 , -0.7487035 , -0.02705123,
        0.13991089,  0.12166452,  0.35100767, -0.74050283,  0.795505  ,
       -0.79394686,  0.2705063 , -0.28195274,  0.17865415,  1.9374261 ,
        1.1716335 ,  1.7149591 , -0.5036105 ,  1.1399124 , -0.65129805,
       -1.0136079 ,  1.3746369 , -0.65427816, -1.6095556 ,  1.8658707 ,
        1.3985107 , -0.2580243 , -1.3835248 , -0.25483572, -0.56031066,
        1.3600457 ,  0.9171877 , -0.47725934,  0.17532575, -0.9789257 ,
       -0.14677837, -1.64347   , -1.0328629 , -1.1871725 ,  0.5564566 ,
        0.6419509 ,  0.71194434, -0.65208477,  2.0065413 , -0.43398145,
        0.21658543, -0.33767387,  1.6739242 ,  0.34716773,  0.6697355 ,
       -1.0252314 ,  0.54468614,  0.20658517, -0.74691606, -1.624849  ,
        0.21332638, -0.27469468, -0.02164643,  1.773442  , -0.90539557,
       -0.6908149 ,  0.47567275, -0.57414037,  2.794381  ,  0.42

In [ ]:
#can use tsne to reduce dimension and make visualization